# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [ ]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,roma,43.2128,-75.4557,33.37,97,100,5.75,US,1738644755
1,1,castillos,-34.1667,-53.8333,72.01,91,3,7.94,UY,1738644756
2,2,dahlonega,34.5326,-83.9849,49.93,82,0,3.74,US,1738644481
3,3,harper,4.3750,-7.7169,77.18,87,95,2.48,LR,1738644758
4,4,la passe,45.5549,-0.8967,40.30,77,0,4.81,FR,1738644760


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
city_data_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df.loc[(city_data_df["Max Temp"] >= 60.0) & (city_data_df["Cloudiness"] <= 30.0) & (city_data_df["Humidity"] <= 60),:]

# Drop any rows with null values
ideal_weather = ideal_weather.dropna(how="any")

# Display sample data
ideal_weather.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,stung treng,13.5259,105.9683,90.61,33,10,6.26,KH,1738644778
22,22,georgetown,5.4112,100.3354,89.53,52,20,8.05,MY,1738644610
33,33,cabo san lucas,22.8909,-109.9124,65.01,45,4,6.60,MX,1738644584
46,46,port augusta,-32.5000,137.7667,91.08,39,1,23.67,AU,1738644810
58,58,gereida,11.2811,25.1414,65.08,27,7,10.69,SD,1738644823


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
20,stung treng,KH,13.5259,105.9683,33,
22,georgetown,MY,5.4112,100.3354,52,
33,cabo san lucas,MX,22.8909,-109.9124,45,
46,port augusta,AU,-32.5000,137.7667,39,
58,gereida,SD,11.2811,25.1414,27,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "limit": 10,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    long = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{lat},{radius}"
    params["bias"] = f"proximity:{long},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
stung treng - nearest hotel: សណ្ធាគារសេកុងស្គារ
georgetown - nearest hotel: Page 63 hostel
cabo san lucas - nearest hotel: Comfort Rooms
port augusta - nearest hotel: Hotel Commonwealth
gereida - nearest hotel: No hotel found
howrah - nearest hotel: Sun India Guest House
baise city - nearest hotel: 百色宾馆
mukalla - nearest hotel: فندق الجابري
san jorge - nearest hotel: Hotel Monumental
pattani - nearest hotel: No hotel found
margaret river - nearest hotel: Margaret River Hotel
ishigaki - nearest hotel: アートホテル石垣島
irymple - nearest hotel: Chaffey Motor Inn
al ghayzah - nearest hotel: فندق تاج العرب
kish - nearest hotel: هتل فارابی
bokoro - nearest hotel: No hotel found
barmer - nearest hotel: Residence Inn
cooma - nearest hotel: No hotel found
bubaque - nearest hotel: Chez Julio
alice springs - nearest hotel: Aurora Alice Springs
pathein - nearest hotel: Taan Taan Ta
mildura - nearest hotel: Chaffey Motor Inn
bardai - nearest hotel: No hotel found
villa ojo de agua - 

,City,Country,Lat,Lng,Humidity,Hotel Name
20,stung treng,KH,13.5259,105.9683,33,សណ្ធាគារសេកុងស្គារ
22,georgetown,MY,5.4112,100.3354,52,Page 63 hostel
33,cabo san lucas,MX,22.8909,-109.9124,45,Comfort Rooms
46,port augusta,AU,-32.5000,137.7667,39,Hotel Commonwealth
58,gereida,SD,11.2811,25.1414,27,No hotel found
64,howrah,IN,22.5892,88.3103,41,Sun India Guest House
87,baise city,CN,23.8997,106.6133,35,百色宾馆
125,mukalla,YE,14.5425,49.1242,21,فندق الجابري
134,san jorge,AR,-31.8962,-61.8598,24,Hotel Monumental
176,pattani,TH,6.7500,100.4167,50,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)
# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)